In [1]:
import pandas as pd
import numpy as np
import os
from transformers import pipeline

/home/ptyagi/miniconda3/envs/thesis_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
replies = pd.read_csv('/work/ptyagi/ClimateVisions/Tweet_Replies/Isaac_replies/replies_translated.csv')

In [3]:
replies.head()

,Unnamed: 0,id,conversation_id,created_at,text
0,1,1120457284268756992,1120240313484959744,2019-04-22T22:40:08.000Z,exactly breaking the law this is what those wo...
1,2,1120263796952567808,1120240313484959744,2019-04-22T09:51:17.000Z,what is your comment
2,3,1120261379427115010,1120240313484959744,2019-04-22T09:41:40.000Z,in the end these are untalented lobbyists whos...
3,4,1120260145362214912,1120240313484959744,2019-04-22T09:36:46.000Z,who is going to believe this organisations thi...
4,5,1120258615678525440,1120240313484959744,2019-04-22T09:30:42.000Z,and i did give both amnety and hws statements ...


In [4]:
replies['created_at'] = pd.to_datetime(replies['created_at'])

In [5]:
replies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3795245 entries, 0 to 3795244
Data columns (total 5 columns):
 #   Column           Dtype              
---  ------           -----              
 0   Unnamed: 0       int64              
 1   id               int64              
 2   conversation_id  int64              
 3   created_at       datetime64[ns, UTC]
 4   text             object             
dtypes: datetime64[ns, UTC](1), int64(3), object(1)
memory usage: 144.8+ MB


In [6]:
replies.head()

,Unnamed: 0,id,conversation_id,created_at,text
0,1,1120457284268756992,1120240313484959744,2019-04-22 22:40:08+00:00,exactly breaking the law this is what those wo...
1,2,1120263796952567808,1120240313484959744,2019-04-22 09:51:17+00:00,what is your comment
2,3,1120261379427115010,1120240313484959744,2019-04-22 09:41:40+00:00,in the end these are untalented lobbyists whos...
3,4,1120260145362214912,1120240313484959744,2019-04-22 09:36:46+00:00,who is going to believe this organisations thi...
4,5,1120258615678525440,1120240313484959744,2019-04-22 09:30:42+00:00,and i did give both amnety and hws statements ...


In [7]:
replies_feb_2019 = replies[(replies['created_at'].dt.month == 2) & (replies['created_at'].dt.year == 2019)].copy()

In [8]:
replies_feb_2019.drop(columns='Unnamed: 0', inplace=True)

In [9]:
replies_feb_2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26961 entries, 36 to 3496374
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               26961 non-null  int64              
 1   conversation_id  26961 non-null  int64              
 2   created_at       26961 non-null  datetime64[ns, UTC]
 3   text             26931 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 1.0+ MB


In [10]:
replies_feb_2019.sort_values(by='created_at', ascending=True, inplace=True)

In [11]:
replies_feb_2019.head()

,id,conversation_id,created_at,text
106617,1091124425456578562,1090610792259600384,2019-02-01 00:01:49+00:00,you did not travel there by plane
107817,1091124695829790720,1090922764586733571,2019-02-01 00:02:54+00:00,looks like this phenomenon has always happened...
107035,1091124765945819137,1091100637566099456,2019-02-01 00:03:10+00:00,i took it for a drive nice car i am not a mass...
3492757,1091124830257274880,1090869895598915584,2019-02-01 00:03:26+00:00,Venn diagram:
3492637,1091125087212777473,1091100637566099456,2019-02-01 00:04:27+00:00,Nee gethu Pannu AOC!


In [12]:
replies_feb_2019.tail()

,id,conversation_id,created_at,text
3495643,1101270493754208259,1100155040839487489,2019-02-28 23:58:40+00:00,U dumb fucktard
136715,1101270588180582402,1101265867353018368,2019-02-28 23:59:03+00:00,shriver unaware we are already importing much ...
131281,1101270664588288000,1098626225201922049,2019-02-28 23:59:21+00:00,keep laptops amp all wireless fa away from u a...
136870,1101270692002217984,1101215748305944576,2019-02-28 23:59:28+00:00,actually it makes you more easy to dismiss kee...
137278,1101270726844264448,1101176583912599552,2019-02-28 23:59:36+00:00,thank you priya now i just have to write it


In [13]:
tweet_ids = replies_feb_2019['conversation_id'].unique()

In [14]:
len(tweet_ids)

4094

In [15]:
import re

def extract_tweet_id(filename):
    match = re.search(r'txt_(\d+)_', filename)
    return match.group(1) if match else None

In [16]:
tweet_texts = {}
tweets_directory = '/work/ptyagi/ClimateVisions/Tweet_Text/2019/02_February'
extracted_tweet_ids = []

for filename in os.listdir(tweets_directory):
    tweet_id = extract_tweet_id(filename)
    
    if tweet_id:
        extracted_tweet_ids.append(int(tweet_id))
        file_path = os.path.join(tweets_directory, filename)
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                tweet_texts[int(tweet_id)] = file.read()

In [17]:
len(extracted_tweet_ids)

57973

In [18]:
tweet_replies_feb_2019 = replies_feb_2019.copy()

In [19]:
tweet_replies_feb_2019['tweet_text'] = replies_feb_2019['conversation_id'].map(tweet_texts)


In [20]:
tweet_replies_feb_2019

,id,conversation_id,created_at,text,tweet_text
106617,1091124425456578562,1090610792259600384,2019-02-01 00:01:49+00:00,you did not travel there by plane,NaN
107817,1091124695829790720,1090922764586733571,2019-02-01 00:02:54+00:00,looks like this phenomenon has always happened...,NaN
107035,1091124765945819137,1091100637566099456,2019-02-01 00:03:10+00:00,i took it for a drive nice car i am not a mass...,NaN
3492757,1091124830257274880,1090869895598915584,2019-02-01 00:03:26+00:00,Venn diagram:,NaN
3492637,1091125087212777473,1091100637566099456,2019-02-01 00:04:27+00:00,Nee gethu Pannu AOC!,NaN
...,...,...,...,...,...
3495643,1101270493754208259,1100155040839487489,2019-02-28 23:58:40+00:00,U dumb fucktard,BREAKING NEWS: Greta Thunberg wird uns bei uns...
136715,1101270588180582402,1101265867353018368,2019-02-28 23:59:03+00:00,shriver unaware we are already importing much ...,Time for some climate change chat #bbcqt https...
131281,1101270664588288000,1098626225201922049,2019-02-28 23:59:21+00:00,keep laptops amp all wireless fa away from u a...,Join me and climate kitty for a @reddit AMA on...
136870,1101270692002217984,1101215748305944576,2019-02-28 23:59:28+00:00,actually it makes you more easy to dismiss kee...,"For an embarrassingly long time, Dems used “sc..."


In [21]:
1090610792259600384 in extracted_tweet_ids

False

In [22]:
tweet_replies_feb_2019.dropna(subset='tweet_text', inplace=True)

In [23]:
len(tweet_replies_feb_2019['conversation_id'].unique())

1776

In [24]:
tweet_replies_feb_2019.head()

,id,conversation_id,created_at,text,tweet_text
2897336,1091463512109563904,1091461450076430336,2019-02-01 22:29:14+00:00,"Climate is a cultural issue, says an expert. I...",Piola. Como las mamás: Antes los veranos en Sa...
3494624,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...
2673599,1092175359326539781,1092171401648369665,2019-02-03 21:37:51+00:00,In addition to the considerable amounts of ele...,Also. Frage an die #Grünsozialisten. Wenn 2018...
2673598,1092175661656141824,1092171401648369665,2019-02-03 21:39:04+00:00,"For future e-cars, charging stations in compan...",Also. Frage an die #Grünsozialisten. Wenn 2018...
2897754,1092518025574731778,1092116835330416645,2019-02-04 20:19:29+00:00,Many people who go to the forest of Aragon do ...,¿How can the wetlands fight the climate change...


In [25]:
tweet_replies_feb_2019.query("conversation_id == 1092171401648369665")['text'].to_list()

['In addition to the considerable amounts of electricity that are already being exported abroad, new gas-fired power plants in Germany have been shut down because the operators prefer to keep dirty lignite-fired power plants running.',
 'For future e-cars, charging stations in company car parks are ideal, where the cars can be charged with solar energy during the day between the start of work and the end of the day.']

In [26]:
tweet_replies_feb_2019.query("conversation_id == 1092171401648369665")['tweet_text'].to_list()

['Also. Frage an die #Grünsozialisten. Wenn 2018 D Netto Stromexporteur war und jetzt von 2020-2030 sowohl #Kohle als Kernstrom Weg fällt, woher beziehen wir dann den Strom für den obligatorischen #Tesla ?#Klimawandel #climatechange #climatestrike #FridaysForFuture https://t.co/DLwKWmD1pl',
 'Also. Frage an die #Grünsozialisten. Wenn 2018 D Netto Stromexporteur war und jetzt von 2020-2030 sowohl #Kohle als Kernstrom Weg fällt, woher beziehen wir dann den Strom für den obligatorischen #Tesla ?#Klimawandel #climatechange #climatestrike #FridaysForFuture https://t.co/DLwKWmD1pl']

In [27]:
tweet_replies_feb_2019.to_csv('/work/ptyagi/masterthesis/tmp/tweet_replies_feb_2019.csv')

In [28]:
classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", device=2)

/home/ptyagi/miniconda3/envs/thesis_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
predictions = classifier(tweet_replies_feb_2019['tweet_text'].tolist(), batch_size=8)

In [41]:
predictions[:5] # type: ignore

[{'label': 'es', 'score': 0.9931243062019348},
 {'label': 'en', 'score': 0.7797765135765076},
 {'label': 'de', 'score': 0.9951056241989136},
 {'label': 'de', 'score': 0.9951056241989136},
 {'label': 'es', 'score': 0.9915847778320312}]

In [43]:
sorted(predictions, key=lambda x: x['score'])[:10] # type: ignore

[{'label': 'nl', 'score': 0.10599808394908905},
 {'label': 'nl', 'score': 0.10765262693166733},
 {'label': 'nl', 'score': 0.10765262693166733},
 {'label': 'nl', 'score': 0.10896977037191391},
 {'label': 'nl', 'score': 0.10896977037191391},
 {'label': 'nl', 'score': 0.10896977037191391},
 {'label': 'nl', 'score': 0.10896977037191391},
 {'label': 'nl', 'score': 0.10896983742713928},
 {'label': 'nl', 'score': 0.10896983742713928},
 {'label': 'nl', 'score': 0.10896983742713928}]

In [44]:
tweet_replies_feb_2019['tweet_lang'] = [pred['label'] for pred in predictions] # type: ignore
tweet_replies_feb_2019['lang_confidence'] = [pred['score'] for pred in predictions] # type: ignore

In [49]:
tweet_replies_feb_2019.head()

,id,conversation_id,created_at,text,tweet_text,tweet_lang,lang_confidence
2897336,1091463512109563904,1091461450076430336,2019-02-01 22:29:14+00:00,"Climate is a cultural issue, says an expert. I...",Piola. Como las mamás: Antes los veranos en Sa...,es,0.993124
3494624,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.779777
2673599,1092175359326539781,1092171401648369665,2019-02-03 21:37:51+00:00,In addition to the considerable amounts of ele...,Also. Frage an die #Grünsozialisten. Wenn 2018...,de,0.995106
2673598,1092175661656141824,1092171401648369665,2019-02-03 21:39:04+00:00,"For future e-cars, charging stations in compan...",Also. Frage an die #Grünsozialisten. Wenn 2018...,de,0.995106
2897754,1092518025574731778,1092116835330416645,2019-02-04 20:19:29+00:00,Many people who go to the forest of Aragon do ...,¿How can the wetlands fight the climate change...,es,0.991585


In [56]:
tweet_replies_feb_2019_en = tweet_replies_feb_2019.query("tweet_lang == 'en'").copy()

In [57]:
tweet_replies_feb_2019_en.dropna(subset='text', inplace=True)

In [73]:
tweet_replies_feb_2019_en.to_csv('/work/ptyagi/masterthesis/tmp/tweet_replies_feb_2019_en.csv')

In [59]:
tweet_replies_feb_2019_en

,id,conversation_id,created_at,text,tweet_text,tweet_lang,lang_confidence
3494624,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.779777
124683,1092776979567468546,1092776506106015746,2019-02-05 13:28:29+00:00,eat local and grow local the ecosystem is buil...,Given how what we eat (demand) determines what...,en,0.896970
3494625,1092835036037087232,1092784983998189568,2019-02-05 17:19:11+00:00,TUMP,New: @MediaLSE PhD candidate Rodrigo Muñoz-Gon...,en,0.890914
124685,1093139326496636928,1093132427810271233,2019-02-06 13:28:19+00:00,the sky is falling the sky in falling the corr...,See how Canada is addressing #ClimateChange th...,en,0.800252
124687,1094072089672175622,1094070997135822848,2019-02-09 03:14:47+00:00,you are beyond demented the utter presumptuous...,You can kiss your beer goodbye if climate chan...,en,0.872495
...,...,...,...,...,...,...,...
136871,1101270343597998080,1101215748305944576,2019-02-28 23:58:05+00:00,i do not see them moving over is there a repub...,"For an embarrassingly long time, Dems used “sc...",en,0.891376
136715,1101270588180582402,1101265867353018368,2019-02-28 23:59:03+00:00,shriver unaware we are already importing much ...,Time for some climate change chat #bbcqt https...,en,0.425586
131281,1101270664588288000,1098626225201922049,2019-02-28 23:59:21+00:00,keep laptops amp all wireless fa away from u a...,Join me and climate kitty for a @reddit AMA on...,en,0.907689
136870,1101270692002217984,1101215748305944576,2019-02-28 23:59:28+00:00,actually it makes you more easy to dismiss kee...,"For an embarrassingly long time, Dems used “sc...",en,0.891376


In [50]:
emotion_classifier = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion-latest", device=2)

In [60]:
emotion_predictions = emotion_classifier(tweet_replies_feb_2019_en['text'].tolist(), batch_size=8)

In [61]:
emotion_predictions[:5] # type: ignore

[{'label': 'anger', 'score': 0.9626848101615906},
 {'label': 'optimism', 'score': 0.9835257530212402},
 {'label': 'anger', 'score': 0.3523235321044922},
 {'label': 'optimism', 'score': 0.8389987945556641},
 {'label': 'anger', 'score': 0.9850420355796814}]

In [70]:
tweet_replies_feb_2019_en['reply_emotion_xlm_base_emotion'] = [pred['label'] for pred in emotion_predictions] # type: ignore
tweet_replies_feb_2019_en['emotion_confidence_xlm_base_emotion'] = [pred['score'] for pred in emotion_predictions] # type: ignore

In [71]:
tweet_replies_feb_2019_en.head()

,id,conversation_id,created_at,text,tweet_text,tweet_lang,lang_confidence,reply_emotion,emotion_confidence,reply_emotion_xlm_base_emotion,emotion_confidence_xlm_base_emotion
3494624,1091717834860826624,1091717129995542528,2019-02-02 15:19:49+00:00,Go Trump!!,Energy Insider: Exploring the impact of winter...,en,0.779777,anger,0.962685,anger,0.962685
124683,1092776979567468546,1092776506106015746,2019-02-05 13:28:29+00:00,eat local and grow local the ecosystem is buil...,Given how what we eat (demand) determines what...,en,0.896970,optimism,0.983526,optimism,0.983526
3494625,1092835036037087232,1092784983998189568,2019-02-05 17:19:11+00:00,TUMP,New: @MediaLSE PhD candidate Rodrigo Muñoz-Gon...,en,0.890914,anger,0.352324,anger,0.352324
124685,1093139326496636928,1093132427810271233,2019-02-06 13:28:19+00:00,the sky is falling the sky in falling the corr...,See how Canada is addressing #ClimateChange th...,en,0.800252,optimism,0.838999,optimism,0.838999
124687,1094072089672175622,1094070997135822848,2019-02-09 03:14:47+00:00,you are beyond demented the utter presumptuous...,You can kiss your beer goodbye if climate chan...,en,0.872495,anger,0.985042,anger,0.985042


In [72]:
print(tweet_replies_feb_2019_en.query("conversation_id == 1092776506106015746")['text'].to_list())
print(tweet_replies_feb_2019_en.query("conversation_id == 1092776506106015746")['reply_emotion_xlm_base_emotion'].to_list())

['eat local and grow local the ecosystem is built like that and would continue to thrive']
['optimism']
